# Pair Programming 30/05 Test estadísticos 4 - Tatiana y Guada

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.multivariate.manova import MANOVA
from sklearn.preprocessing import StandardScaler

plt.rcParams["figure.figsize"] = (10,8) 

# Ejercicio 1
-Estandarizar las columnas de las variables predictoras


In [2]:
df = pd.read_csv("listings_limpio_2.csv", index_col=0)
df.head(2)

,neighbourhood_group,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm
0,Chamartín,Private room,3.021213,1,81,0.57,1,149,3
1,Latina,Private room,2.661819,4,33,0.38,2,131,0


In [3]:
#Estas son nuestras columnas que queremos estandarizar
df_estandarizar = df[["minimum_nights", "number_of_reviews", "reviews_per_month", "calculated_host_listings_count", "availability_365", "number_of_reviews_ltm"]]

In [4]:
st = StandardScaler()
df_std = pd.DataFrame(st.fit_transform(df_estandarizar), columns=df_estandarizar.columns)
df_std.head()

,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm
0,-0.176920,0.647995,-0.561126,-0.382089,0.006434,-0.269185
1,-0.094650,-0.047665,-0.698318,-0.346261,-0.122452,-0.493411
2,0.207007,-0.525932,0.000012,-0.202945,1.166410,-0.493411
3,-0.067227,-0.323031,-0.857171,-0.382089,0.135321,-0.194442
4,-0.149497,1.705977,-0.185653,-0.382089,1.402701,-0.119700


In [5]:
columnas = df_std.columns
df[columnas] = df_std
df.head()

,neighbourhood_group,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm
0,Chamartín,Private room,3.021213,-0.176920,0.647995,-0.561126,-0.382089,0.006434,-0.269185
1,Latina,Private room,2.661819,-0.094650,-0.047665,-0.698318,-0.346261,-0.122452,-0.493411
2,Arganzuela,Entire home/apt,2.932182,0.207007,-0.525932,0.000012,-0.202945,1.166410,-0.493411
3,Centro,Entire home/apt,3.249172,-0.067227,-0.323031,-0.857171,-0.382089,0.135321,-0.194442
4,Arganzuela,Private room,2.622480,-0.149497,1.705977,-0.185653,-0.382089,1.402701,-0.119700


# Ejercicio 2
-Cread un ANOVA, siendo la variable respuesta price y las variables predictoras el resto de las columnas del dataframe

In [8]:
# iniciamos nuestro anova. Recordemos que la variable respuesta es el precio de las casas (price)
lm = ols('price ~ + neighbourhood_group + room_type + minimum_nights + + calculated_host_listings_count + number_of_reviews + reviews_per_month + availability_365 + number_of_reviews_ltm', data=df).fit()
sm.stats.anova_lm(lm)

,df,sum_sq,mean_sq,F,PR(>F)
neighbourhood_group,20.0,281.582957,14.079148,112.102783,0.000000e+00
room_type,3.0,720.887259,240.295753,1913.313437,0.000000e+00
minimum_nights,1.0,0.136758,0.136758,1.088908,2.967279e-01
calculated_host_listings_count,1.0,0.485087,0.485087,3.862423,4.939507e-02
number_of_reviews,1.0,9.811104,9.811104,78.119218,1.062384e-18
reviews_per_month,1.0,0.047197,0.047197,0.375801,5.398669e-01
availability_365,1.0,0.499887,0.499887,3.980268,4.605204e-02
number_of_reviews_ltm,1.0,0.016545,0.016545,0.131736,7.166429e-01
Residual,17156.0,2154.646417,0.125591,NaN,NaN


👩🏻👩🏻‍🦰💬

El resultado de la columna PR(>F) se refiere al *p-valor* con el que se determina si se rechaza la hipótesis nula (recordamos también que la hipótesis nula era aquello contrario a lo que queríamos predecir), siendo ésta que la variable no tiene efecto sobre la variable respuesta. Si el valor p es menor que o igual a 0.05. Las columnas con ese valor son todas menos "reviews_per_month" y "number_of_reviews_ltm". En este caso, debemos rechazar la hipótesis nula y concluimos que esa variable si influye en el precio de las casas. 


In [9]:
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.320
Model:                            OLS   Adj. R-squared:                  0.319
Method:                 Least Squares   F-statistic:                     278.3
Date:                Tue, 31 May 2022   Prob (F-statistic):               0.00
Time:                        15:17:11   Log-Likelihood:                -6542.8
No. Observations:               17186   AIC:                         1.315e+04
Df Residuals:                   17156   BIC:                         1.338e+04
Df Model:                          29                                         
Covariance Type:            nonrobust                                         
================================================================================================================
                                                   coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------
Intercept                                        3.2026      0.012    271.465      0.000       3.179       3.226
neighbourhood_group[T.Barajas]                   0.0937      0.033      2.851      0.004       0.029       0.158
neighbourhood_group[T.Carabanchel]              -0.0667      0.019     -3.594      0.000      -0.103      -0.030
neighbourhood_group[T.Centro]                    0.0874      0.012      7.139      0.000       0.063       0.111
neighbourhood_group[T.Chamartín]                 0.0978      0.020      4.968      0.000       0.059       0.136
neighbourhood_group[T.Chamberí]                  0.1040      0.016      6.499      0.000       0.073       0.135
neighbourhood_group[T.Ciudad Lineal]            -0.0061      0.020     -0.315      0.753      -0.044       0.032
neighbourhood_group[T.Fuencarral - El Pardo]     0.0424      0.025      1.720      0.085      -0.006       0.091
neighbourhood_group[T.Hortaleza]                 0.1055      0.022      4.721      0.000       0.062       0.149
neighbourhood_group[T.Latina]                   -0.1181      0.020     -5.920      0.000      -0.157      -0.079
neighbourhood_group[T.Moncloa - Aravaca]         0.0398      0.020      2.017      0.044       0.001       0.079
neighbourhood_group[T.Moratalaz]                -0.1335      0.035     -3.852      0.000      -0.201      -0.066
neighbourhood_group[T.Puente de Vallecas]       -0.1360      0.020     -6.785      0.000      -0.175      -0.097
neighbourhood_group[T.Retiro]                    0.0362      0.019      1.952      0.051      -0.000       0.073
neighbourhood_group[T.Salamanca]                 0.1317      0.016      8.490      0.000       0.101       0.162
neighbourhood_group[T.San Blas - Canillejas]     0.3544      0.021     16.845      0.000       0.313       0.396
neighbourhood_group[T.Tetuán]                    0.0013      0.017      0.075      0.940      -0.033       0.036
neighbourhood_group[T.Usera]                    -0.0988      0.025     -3.888      0.000      -0.149      -0.049
neighbourhood_group[T.Vicálvaro]                 0.0757      0.045      1.687      0.092      -0.012       0.164
neighbourhood_group[T.Villa de Vallecas]        -0.0495      0.042     -1.179      0.238      -0.132       0.033
neighbourhood_group[T.Villaverde]               -0.2016      0.030     -6.672      0.000      -0.261      -0.142
room_type[T.Hotel room]                          0.1256      0.030      4.229      0.000       0.067       0.184
room_type[T.Private room]                       -0.4338      0.006    -73.303      0.000      -0.445      -0.422
room_type[T.Shared room]                        -0.5775      0.024    -24.472      0.000      -0.624      -0.531
minimum_nights                  

👩🏻👩🏻‍🦰💬

Interpretemos estos resultados:

    - `coef`: es el coeficiente que representa los cambios medios en la variable respuesta para una unidad de cambio en la variable predictora mientras se mantienen constantes el resto de las variables predictoras. Es la tasa de cambio, nos indica lo que cambia la variable respuesta (en nuestro caso el precio) por cada unidad que aumenta la variable predictora. En nuestros resultados, todas las columnas han salido negativas, excepto "availability_365", aunque están cerca de cero. Esto quiere decir que la única variable que cambia considerablemente al aumentar el precio sería availability_365.

    - `std err`: es el error estándar del coeficiente. Se usa para medir la precisión de la estimación del coeficiente. Cuanto menor sea el error estándar, más precisa será la estimación. En nuestro dataset, sus valores son bajos, por lo que la precisión es buena.

    - `t` : es el resultado de dividir el coeficiente entre su error estándar. Como era de esperar tras ver el coeficiente en el primer punto, estos valores son más bajos, llegando al punto de que la variable "number_of_reviews" llega a estar en más de -3.

    - `P`: Para determinar si cualquiera de las diferencias entre las medias es estadísticamente significativa, deberemos comparar el valor p con el nivel de significancia para evaluar la hipótesis nula. En "number_of_reviews" es 0, por lo que se acepta la hipótesis nula, mientras que en el resto de variable dista de ser 0, aunque en el caso de "minimum_nights" no se aleja demasiado.

    - `R-squared`: el porcentaje de la variación de la variable respuesta está explicado por nuestro modelo. Es decir, nuestros variables predictoras explican el   0.2% de la variación en los precios de las casas.